# Working with `Community`s

the `Community` object is geosnap's central data structure. A `Community` is a dataset that stores information about a collection of neighborhoods over several time periods, including each neighborhood's physical, socioeconomic, and demographic attributes and its demarcated boundaries. Under the hood, each `Community` is simply a long-form geopandas geodataframe with some associated metadata. 

If you're working with built-in data, you instantiate a `Community` by choosing the constructor for your dataset and passing either a boundary (geodataframe) or a selection filter that defines the study area. The selection filter can be either a `GeoDataFrame` boundary or a set of [FIPS](https://www.policymap.com/2012/08/tips-on-fips-a-quick-guide-to-geographic-place-codes-part-iii/) codes. Boundary queries are often more convenient but they are  more expensive to compute and will take longer to construct.

When constructing `Community`s from fips codes, the constructor has arguments for state, county, msa, or list of any arbitrary fips codes. If more than one of these arguments is passed, geosnap will use the union. This means that each level of the hierarchy is available for convenience but you are free to mix and match msas, states, counties, and even single tracts to create your study region of choice

If you're working with your own data, you instantiate a `Community` by passing a list of geodataframes (or a single long-form).

In [1]:
from geosnap import Community

## Create a `Community` from built-in census data

The quickest and easiest method for getting started is to instantiate a Community using the built-in census data. To do so, you use the `Community.from_census` constructor:

In [2]:
# this will create a new community using data from Washington DC (which is fips code 11)
dc = Community.from_census(state_fips="11")

/Users/the-gaboardi/geosnap/geosnap/_data.py:220: UserWarning: streaming remote data. Use `geosnap.io.store_census() to store the data locally for better performance
  warn(
/Users/the-gaboardi/geosnap/geosnap/_data.py:251: UserWarning: streaming remote data. Use `geosnap.io.store_census() to store the data locally for better performance
  warn(
/Users/the-gaboardi/geosnap/geosnap/_data.py:283: UserWarning: streaming remote data. Use `geosnap.io.store_census() to store the data locally for better performance
  warn(


Note that when using `Community.from_census`, the resulting community has *unharmonized* tract boundaries, meaning that the tracts are different for each decade

To access the underlying data from a `Community`, simply call its `gdf` attribute which returns a geodataframe

In [3]:
dc.gdf.head()

,geoid,n_mexican_pop,n_cuban_pop,n_puerto_rican_pop,n_total_housing_units,n_vacant_housing_units,n_occupied_housing_units,n_owner_occupied_housing_units,n_renter_occupied_housing_units,n_white_persons,...,p_irish_born_pop,p_italian_born_pop,p_poverty_rate_children,p_poverty_rate_hispanic,p_russian_born_pop,p_scandanavian_born_pop,p_scandanavian_pop,n_total_pop_sample,p_female_labor_force,p_black_persons
53214,11001001600,4409.0,2.0,7.0,1667.0,43.0,1624.0,1458.0,166.0,114423.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53215,11001001500,5430.0,11.0,15.0,2309.0,78.0,2231.0,1898.0,333.0,4694102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53216,11001001701,2868.0,5.0,22.0,1287.0,50.0,1237.0,619.0,618.0,31727.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53217,11001001801,694.0,5.0,28.0,10.0,1.0,9.0,0.0,9.0,35813.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53218,11001001702,2516.0,6.0,12.0,1086.0,43.0,1043.0,712.0,331.0,48917.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


You can easily create figures of time series plots from geosnap `Community` objects using the `plot_timeseries` class method. `plot_timeseries` uses `proplot`, a lightweight `matplotlib` wrapper with its own feature set. It also uses [`contextily`](https://github.com/geopandas/contextily) to (optionally) provide a nice basemap

In [4]:
dc.plot_timeseries(
    "p_nonhisp_white_persons",
    scheme="quantiles",
    cmap="Blues",
    k=6,
    alpha=0.7,
    years=[1990, 2000, 2010],
)

/Users/the-gaboardi/miniconda3/envs/geosnap/lib/python3.8/site-packages/proplot/config.py:1454: ProPlotWarning: Rebuilding font cache.


AttributeError: module 'matplotlib.font_manager' has no attribute '_fmcache'

`plot_timeseries` default behavior is to graph every year in the provided `Community` object in chronological order with OpenStreetMaps.Mapnik from contextily as the basemap. If you pass the list `years=[]` , then `plot_timeseries` will graph in the order that the list was passed. You can also specify another map provider for contextily, or disable it by passing `False` with the parameter`ctxmap`.

The `save_fig` parameter in `plot_timeseries` tells the function to save the resulting plot with the specified name and path. Note that if the file extension is not specified then the image will be saved as a `.pdf`, and that the path specified must already exist.

## Create a `Community` from Longitudinal Employment-Household Dynamics

You can also create a `Community` from block-level LEHD census data. Unlike the decennial census, LEHD data are annual and contain information about the "workplace area characteristics" ("wac") and "residence area characteristics" ("rac") of employees. "wac" datasets contain information about where employees work, while "rac" datasets contiain information about where they live. Apart from information about the race, skill level, and income of emmployees in eaech census block, LEHD data also count the number of workers in each NAICS 2-digit industry category.

If you use the `Community.from_lodes` constructor, you can collect data from 2000 to 2015

In [ ]:
delaware = Community.from_lodes(state_fips="10", years=[2014, 2015])

In [ ]:
delaware.gdf.head()

In [ ]:
delaware.gdf[delaware.gdf.year == 2015].plot(
    column="total_employees", scheme="quantiles", k=9
)

## Create a `Community` from a longitudinal database

To instantiate a `Community` from a longitudinal database, you must first register the database with geosnap using either `store_ltdb` or `store_ncdb`. Once the data are available in datasets, you can call `Community.from_ltdb` and `Community.from_ncdb`

### LTDB using fips codes

I don't know the Riverside MSA fips code by heart, so I'll slice through the `msas` dataframe in the data store to find it

In [5]:
from geosnap import datasets

In [6]:
datasets.msas()[datasets.msas().name.str.startswith("Riverside")]

,geoid,name,type,geometry
728,40140,"Riverside-San Bernardino-Ontario, CA",Metro Area,"POLYGON ((-117.67302 33.87104, -117.67481 33.8..."


In [7]:
riverside = Community.from_ltdb(msa_fips="40140")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/the-gaboardi/Library/Application Support/geosnap/ltdb.parquet'

In [ ]:
riverside.plot_timeseries("p_poverty_rate", scheme="quantiles", cmap="Blues", k=6)

Instead of passing a fips code, I could use the `boundary` argument to pass the riverside MSA as a geodataframe. This is more computationally expensive because it requires geometric operations, but is more flexible because it allows you to create communities that don't nest into fips hierarchies (like zip codes, census designated places, or non-US data)

### NCDB Using a boundary

In [ ]:
# grab the boundary for Sacramento from libpysal's built-in examples

import geopandas as gpd
from libpysal.examples import load_example

sac = load_example("Sacramento1")
sac = gpd.read_file(sac.get_path("sacramentot2.shp"))

In [ ]:
sacramento = Community.from_ncdb(boundary=sac)

In [ ]:
import contextily as ctx

In [ ]:
sacramento.plot_timeseries(
    "median_household_income",
    scheme="quantiles",
    cmap="Blues",
    k=6,
    alpha=0.6,
    ctxmap=ctx.providers.Stamen.TonerLite,
    years=[1990, 2000, 2010],
)

## Create a `Community` from a list of geodataframes

If you are working outside the US, or if you have data that aren't included in geosnap (like census blocks or zip codes) then you can still create a community using the `Community.from_geodataframes` constructor, which allows you to pass a list of geodataframes that will be concatenated into the single long-form gdf structure that geosnap's analytics expect.

This constructor is typically used in cases where a researcher has several shapefiles for a study area, each of which pertainin to a different time period. In such a case, the user would read each shapefile into a geodataframe and ensure that each has a "time" column that will differentiate each time period from one another in the long-form structure (e.g. if each shapefile is a different decade, then the 1990 shapefile should have a column called "year" in which every observation has a value of 1990). Then, these geodataframes simply need to be passed in a list to the `from_geodataframes` constructor

Here, I'll use `cenpy` to grap population data from two different ACS vintages and combine them into a single community

In [ ]:
from cenpy.products import ACS

In [ ]:
chi13 = ACS(2013).from_place("chicago", variables="B00001_001E")

In [ ]:
chi13["year"] = 2013

In [ ]:
chi17 = ACS(2017).from_place("chicago", variables="B00001_001E")

In [ ]:
chi17["year"] = 2017

In [ ]:
chicago = Community.from_geodataframes([chi13, chi17])

In [ ]:
chicago.plot_timeseries(
    "B00001_001E",
    cmap="Greens",
    scheme="quantiles",
    k=6,
    ctxmap=ctx.providers.Stamen.TonerLite,
    alpha=0.5,
    title='Chicago Population by Census Tract',
)